행정구역 의료기관 현황분석하기

In [1]:
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt


파일을 불러와보자

In [ ]:
data=pd.read_csv('../data4/공공보건의료기관현황.csv', encoding='cp949')
data 

,연번,병원 명,설립형태,근거 법령,관계 행정기관,관계 공공단체,심평원 요양기관번호,종별구분,병상수,소재지 우편번호,주소,홈페이지,대표전화,FAX,비고
0,1,강원도 재활병원,시도립,강원도재활병원설치및운영에관한조례,강원도,해당없음,32200641,병원,165,24227,강원도 춘천시 충열로 142번길 24-16,www.grh.or.kr,033-248-7700,033-248-7723,NaN
1,2,강원도 삼척의료원,특수법인,지방의료원의 설립 및 운영에 관한 법률,보건복지부(강원도),지방의료원,32100060,종합병원,152,25920,강원도 삼척시 오십천로 418,http://ksmc.or.kr,033-572-1141,033-573-8424,NaN
2,3,강원도 영월의료원,특수법인,지방의료원의 설립 및 운영에 관한 법률,보건복지부(강원도),지방의료원,32100078,종합병원,214,26234,강원도 영월군 영월읍 중앙1로 59,http://www.youngwol.org,033-370-9117,033-370-9137,NaN
3,4,강원도 원주의료원,특수법인,지방의료원의 설립 및 운영에 관한 법률,보건복지부(강원도),지방의료원,32100086,종합병원,237,26448,강원도 원주시 서원대로 387(개운동),www.kwmc.or.kr,033-760-4500,033-761-5121,NaN
4,5,강원도 강릉의료원,특수법인,지방의료원의 설립 및 운영에 관한 법률,보건복지부(강원도),지방의료원,32100159,종합병원,137,25535,강원도 강릉시 경강로 2007(남문동 164-1),http://www.gnmc.or.kr,033-646-6910,033-610-1415,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,216,청풍호노인사랑병원,시군구립,제천시 노인전문병원 설치및 운영에 관한 조례,충청북도,해당없음,33280576,요양병원,187,27207,충청북도 제천시 금성면 청풍호로 1147,www.cplove.co.kr,043-646-0808,043-648-5827,NaN
223,217,충주시 노인전문병원,시군구립,충주시노인전문병원설치 및 운영에관한 조례,충청북도,해당없음,33280088,요양병원,300,27442,충청북도 충주시 동량면 대전리 1686-28,www.hyolife.com,043-855-9393,043-855-8866,NaN
224,218,충청북도 충주의료원,특수법인,지방의료원의 설립 및 운영에 관한 법률,충청북도,해당없음,33100063,종합병원,292,27468,충청북도 충주시 안림로 239-50(안림동 134),www.cjmct.or.kr,043-871-0114,043-843-4231,NaN
225,219,영동군립노인전문병원,시군구립,영동군립노인전문병원설치 및 운영조례,충청북도,해당없음,33280681,요양병원,120,29131,충청북도 영동군 영동읍 대학로 290,cmisilveryd.com,043-744-9633,043-742-8275,NaN


주소 데이터 뽑기

주소줄로 데이터 프레임 만들었어!!

In [ ]:
addr=pd.DataFrame(data['주소'].apply(lambda x: x.split()[:2]).tolist(), columns=['시도','군구']) 
#람다 x는 주소 1개를 입력값으로 하겠다. x.공백으로 나누겠다. 리스트에 넣겠다.
#data['주소'].apply(lambda x: x.split()[:2]) 데이터 프레임의 데이터이고, 컬럼명이고 
addr

,시도,군구
0,강원도,춘천시
1,강원도,삼척시
2,강원도,영월군
3,강원도,원주시
4,강원도,강릉시
...,...,...
222,충청북도,제천시
223,충청북도,충주시
224,충청북도,충주시
225,충청북도,영동군


옛날 입력 데이터를 전산화 할 때, 지역명을 제각각 써서 정리해줄 필요가 있어. 잘못된 데이터를 바로 잡을 필요가 있어.
광역시도가 아닌 것을 정리해서 나열해보자.

In [14]:
addr['시도'].unique()

array(['강원도', '경기도', '경기', '경남', '경상남도', '경상북도', '경북', '인천광역시', '대구광역시',
       '전라남도', '대전광역시', '광주광역시', '제주특별자치도', '부산광역시', '전라북도', '충북',
       '서울특별시', '서울시', '부산특별시', '대전시', '충남', '전남', '충청남도', '울산광역시', '전북',
       '충청북도'], dtype=object)

In [5]:
addr[addr['시도']=='창원시']

,시도,군구
27,창원시,의창구
31,창원시,마산합포구3.15대로


27번째 줄 수정, 31번째 줄

In [6]:
addr.iloc[27]=['경상남도','창원시']
addr.iloc[31]=['경상남도','창원시']

수정하니까 창원시가 사라짐.

In [7]:
addr[addr['시도']=='창원시']

,시도,군구


In [8]:
addr[addr['시도']=='경산시']

,시도,군구
47,경산시,경안로


In [9]:
addr.iloc[47]=['경상북도','경산시']

In [10]:
addr[addr['시도']=='경산시']

,시도,군구


In [11]:
addr[addr['시도']=='천안시']

,시도,군구
209,천안시,동남구
210,천안시,동남구


In [12]:
addr.iloc[209]=['충청남도','천안시']
addr.iloc[210]=['충청남도','천안시']

In [13]:
addr[addr['시도']=='천안시']

,시도,군구


데이터 이름 맞춰주자 부산 부르는게 너무 많다.

In [15]:
addr_aliases={'서울시':'서울특별시', 
              '부산특별시':'부산광역시',
              '경기':'경기도',
              '경남':'경상남도',
              '경북':'경상북도',
              '충남':'충청남도',
              '충북':'충청북도',
              '전남':'전라남도',
              '전북':'전북특별자치도',
              '강원도':'강원특별자치도'
              }
addr['시도']=addr['시도'].replace(addr_aliases)
addr['시도'].unique()

array(['강원특별자치도', '경기도', '경상남도', '경상북도', '인천광역시', '대구광역시', '전라남도',
       '대전광역시', '광주광역시', '제주특별자치도', '부산광역시', '전라북도', '충청북도', '서울특별시',
       '대전시', '충청남도', '울산광역시', '전북특별자치도'], dtype=object)

In [21]:
addr['군구'].unique()

array(['춘천시', '삼척시', '영월군', '원주시', '강릉시', '속초시', '정선군', '수원시', '이천시',
       '안성시', '의정부시', '포천시', '파주시', '용인시', '평택시', '시흥시', '여주시', '남양주시',
       '동두천시', '안산시', '부천시', '통영시', '사천시', '창원시', '김해시', '양산시', '거창군',
       '남해군', '의령군', '포항시', '김천시', '안동시', '울진군', '경주시', '구미시', '영주시',
       '상주시', '문경시', '경산시', '의성군', '청도군', '고령군', '칠곡군', '봉화군', '울릉군',
       '부평구', '북구', '순천시', '대덕구', '태백시', '동해시', '화성시', '광산구', '남구', '중구',
       '제주시', '서구', '전주시', '진주시', '청주시', '종로구', '성남시', '동구', '화순군', '강동구',
       '사상구', '달서구', '해운대구', '유성구', '가평군', '양주시', '고양시', '홍천군', '양구군',
       '청원군', '계룡시', '논산시', '함평군', '양평군', '수성구', '달성군', '연수구', '노원구',
       '기장군', '공주시', '강북구', '광진구', '나주시', '창녕군', '목포시', '고흥군', '연제구',
       '동매로', '서초구', '은평구', '중랑구', '강남구', '동작구', '동대문구', '양천구', '성동구',
       '송파구', '울주군', '계양구', '옹진군', '보성군', '광양시', '영광군', '무안군', '진도군',
       '강진군', '곡성군', '여수시', '신안군', '장성군', '완주군', '부안군', '정읍시', '남원시',
       '군산시', '고창군', '진안군', '서귀포시', '천안시', '보령시', '서산시', '서천군', '홍성군',
     

아란13길이 시군구 잘못된거야. 제주 특별자치도

In [22]:
addr[addr['군구']=='아란13길']
addr.iloc[75] =['제주특별자치도','제주시']
addr[addr['군구']=='아란13길']

,시도,군구


시군구 공공의료기관 카운팅할거야
시도군구 하나의 필드로 만들어서 그룹바이하자. 대구 부산 서울 중구 다있어.

In [23]:
addr['시도군구']=addr['시도']+' '+addr['군구']
addr

,시도,군구,시도군구
0,강원특별자치도,춘천시,강원특별자치도 춘천시
1,강원특별자치도,삼척시,강원특별자치도 삼척시
2,강원특별자치도,영월군,강원특별자치도 영월군
3,강원특별자치도,원주시,강원특별자치도 원주시
4,강원특별자치도,강릉시,강원특별자치도 강릉시
...,...,...,...
222,충청북도,제천시,충청북도 제천시
223,충청북도,충주시,충청북도 충주시
224,충청북도,충주시,충청북도 충주시
225,충청북도,영동군,충청북도 영동군


카운트 해보자, count 필드를 미리 만들자.

In [24]:
addr['count']=0
addr

,시도,군구,시도군구,count
0,강원특별자치도,춘천시,강원특별자치도 춘천시,0
1,강원특별자치도,삼척시,강원특별자치도 삼척시,0
2,강원특별자치도,영월군,강원특별자치도 영월군,0
3,강원특별자치도,원주시,강원특별자치도 원주시,0
4,강원특별자치도,강릉시,강원특별자치도 강릉시,0
...,...,...,...,...
222,충청북도,제천시,충청북도 제천시,0
223,충청북도,충주시,충청북도 충주시,0
224,충청북도,충주시,충청북도 충주시,0
225,충청북도,영동군,충청북도 영동군,0


In [26]:
addr_group=addr.groupby(['시도','군구','시도군구'],as_index=False).count()
addr_group

,시도,군구,시도군구,count
0,강원특별자치도,강릉시,강원특별자치도 강릉시,4
1,강원특별자치도,동해시,강원특별자치도 동해시,1
2,강원특별자치도,삼척시,강원특별자치도 삼척시,1
3,강원특별자치도,속초시,강원특별자치도 속초시,1
4,강원특별자치도,양구군,강원특별자치도 양구군,1
...,...,...,...,...
136,충청북도,영동군,충청북도 영동군,1
137,충청북도,제천시,충청북도 제천시,1
138,충청북도,청원군,충청북도 청원군,1
139,충청북도,청주시,충청북도 청주시,4


In [29]:
addr_group.set_index('시도군구',inplace=True)
addr_group

KeyError: "None of ['시도군구'] are in the columns"

In [30]:
addr_group

,시도,군구,count
시도군구,,,
강원특별자치도 강릉시,강원특별자치도,강릉시,4
강원특별자치도 동해시,강원특별자치도,동해시,1
강원특별자치도 삼척시,강원특별자치도,삼척시,1
강원특별자치도 속초시,강원특별자치도,속초시,1
강원특별자치도 양구군,강원특별자치도,양구군,1
...,...,...,...
충청북도 영동군,충청북도,영동군,1
충청북도 제천시,충청북도,제천시,1
충청북도 청원군,충청북도,청원군,1


In [31]:
population=pd.read_excel('../data4/행정구역_시군구_별__성별_인구수_2.xlsx')
population

,시도,군구,총인구수,남자인구수,여자인구수
0,전국,합계,51313912,25558944,25754968
1,서울특별시,소계,9384325,4538354,4845971
2,서울특별시,종로구,139378,67240,72138
3,서울특별시,중구,121322,58651,62671
4,서울특별시,용산구,212175,101793,110382
...,...,...,...,...,...
288,경상남도,거창군,59909,29480,30429
289,경상남도,합천군,41096,20094,21002
290,제주특별자치도,소계,674353,337578,336775
291,제주특별자치도,제주시,490888,245197,245691


공백들이 많아!!!
그냥 기계적으로 공백 날리자. 1. unique 해보고 공백있는지 확인하고, 이름이 같은지 확인하고, 기계 날리자.

In [32]:
population['군구']=population['군구'].apply(lambda x:x.strip())
population['시도']=population['시도'].apply(lambda x:x.strip())

In [37]:
population['시도'].unique()
population['군구'].unique() #출장소라는 것들이 생겼어. 지워야하는데!!
population.loc[population['총인구수']==0]

,시도,군구,총인구수,남자인구수,여자인구수
56,인천광역시,중구영종출장소,0,0,0
57,인천광역시,중구용유출장소,0,0,0
65,인천광역시,서구검단출장소,0,0,0
108,경기도,송탄출장소,0,0,0
109,경기도,안중출장소,0,0,0
121,경기도,풍양출장소,0,0,0
136,경기도,화성시동부출장소,0,0,0
137,경기도,화성시동탄출장소,0,0,0
273,경상남도,사천남양출장소,0,0,0
275,경상남도,장유출장소,0,0,0


In [36]:
addr['시도'].unique()
addr['군구'].unique()



array(['춘천시', '삼척시', '영월군', '원주시', '강릉시', '속초시', '정선군', '수원시', '이천시',
       '안성시', '의정부시', '포천시', '파주시', '용인시', '평택시', '시흥시', '여주시', '남양주시',
       '동두천시', '안산시', '부천시', '통영시', '사천시', '창원시', '김해시', '양산시', '거창군',
       '남해군', '의령군', '포항시', '김천시', '안동시', '울진군', '경주시', '구미시', '영주시',
       '상주시', '문경시', '경산시', '의성군', '청도군', '고령군', '칠곡군', '봉화군', '울릉군',
       '부평구', '북구', '순천시', '대덕구', '태백시', '동해시', '화성시', '광산구', '남구', '중구',
       '제주시', '서구', '전주시', '진주시', '청주시', '종로구', '성남시', '동구', '화순군', '강동구',
       '사상구', '달서구', '해운대구', '유성구', '가평군', '양주시', '고양시', '홍천군', '양구군',
       '청원군', '계룡시', '논산시', '함평군', '양평군', '수성구', '달성군', '연수구', '노원구',
       '기장군', '공주시', '강북구', '광진구', '나주시', '창녕군', '목포시', '고흥군', '연제구',
       '동매로', '서초구', '은평구', '중랑구', '강남구', '동작구', '동대문구', '양천구', '성동구',
       '송파구', '울주군', '계양구', '옹진군', '보성군', '광양시', '영광군', '무안군', '진도군',
       '강진군', '곡성군', '여수시', '신안군', '장성군', '완주군', '부안군', '정읍시', '남원시',
       '군산시', '고창군', '진안군', '서귀포시', '천안시', '보령시', '서산시', '서천군', '홍성군',
     

인덱스로 두개를 합쳐서 작업할거라면 맞는지 확인을 해야해!!!!!